In [4]:
import os
!python -m pip install nltk
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from IPython.display import Audio
import nltk  # we'll use this to split into sentences
import numpy as np

from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


/Users/nshykula/TTScocqui/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
preload_models()

/Users/nshykula/TTScocqui/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [6]:
!export PYTORCH_ENAVLE_MPS_FALLBACK=1
!export SUNO_MPS_ENABLE_CROSS_DEVICE=1
!export SUNO_MPS_ENABLR=True

# Simple Long-Form Generation
We split longer text into sentences using `nltk` and generate the sentences one by one.

In [33]:
script = """
Hey, have you heard about this new text-to-audio model called "Bark"? 
Apparently, it's the most realistic and natural-sounding text-to-audio model 
out there right now. People are saying it sounds just like a real person speaking. 
I think it uses advanced machine learning algorithms to analyze and understand the 
nuances of human speech, and then replicates those nuances in its own speech output. 
It's pretty impressive, and I bet it could be used for things like audiobooks or podcasts. 
In fact, I heard that some publishers are already starting to use Bark to create audiobooks. 
It would be like having your own personal voiceover artist. I really think Bark is going to 
be a game-changer in the world of text-to-audio technology.
""".replace("\n", " ").strip()

In [34]:
sentences = nltk.sent_tokenize(script)

In [35]:
SPEAKER = "v2/en_speaker_6"
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

pieces = []
for sentence in sentences:
    audio_array = generate_audio(sentence, history_prompt=SPEAKER)
    pieces += [audio_array, silence.copy()]


100%|████████████████████████████████████████████████████████████████████████| 15/15 [00:05<00:00,  2.57it/s]


In [ ]:
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)

# $ \\ $

# Advanced Long-Form Generation
Somtimes Bark will hallucinate a little extra audio at the end of the prompt.
We can solve this issue by lowering the threshold for bark to stop generating text. 
We use the `min_eos_p` kwarg in `generate_text_semantic`

In [7]:
GEN_TEMP = 0.6
SPEAKER = "v2/en_speaker_6"
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

pieces = []
for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=0.05,  # this controls how likely the generation is to end
    )

    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]



NameError: name 'sentences' is not defined

In [ ]:
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)

# $ \\ $

# Make a Long-Form Dialog with Bark

### Step 1: Format a script and speaker lookup

In [17]:
speaker_lookup = {"Samantha": "v2/en_speaker_9", "John": "v2/en_speaker_2"}

# Script generated by chat GPT
script = f"""
Samantha:[EROTICA]mmmmmm, I want you inside me.
John:[EROTICA] you know Ill always give you what you want.
Samantha:[EROTICA]Thats why I love you.
John:[EROTICA]Come here, baby.
Samantha:[EROTICA]Yes, sir.
John:[EROTICA]Put your legs around me.
Samantha:[EROTICA]Like this?
John:[EROTICA]Perfect.
Samantha:[EROTICA]Oh, God.
John:[EROTICA]You feel so good.
Samantha:[EROTICA]Dont stop.[fucking].
John:[EROTICA]Never.
Samantha:[EROTICA] Were going to make such beautiful babies together.
John:[EROTICA]Thats our plan, sweetheart.
Samantha:[EROTICA]Oooohhhh...[SEX SFX].
John:[EROTICA]ugh, ugh, you like when I fuck that pussy dont you?
Samantha:[EROTICA]yes! yes! uuh, uh! my tight cunt loves yoour big dick!
"""
script = script.strip().split("\n")
script = [s.strip() for s in script if s]
script

['Samantha:[EROTICA]mmmmmm, I want you inside me.',
 'John:[EROTICA] you know Ill always give you what you want.',
 'Samantha:[EROTICA]Thats why I love you.',
 'John:[EROTICA]Come here, baby.',
 'Samantha:[EROTICA]Yes, sir.',
 'John:[EROTICA]Put your legs around me.',
 'Samantha:[EROTICA]Like this?',
 'John:[EROTICA]Perfect.',
 'Samantha:[EROTICA]Oh, God.',
 'John:[EROTICA]You feel so good.',
 'Samantha:[EROTICA]Dont stop.[fucking].',
 'John:[EROTICA]Never.',
 'Samantha:[EROTICA] Were going to make such beautiful babies together.',
 'John:[EROTICA]Thats our plan, sweetheart.',
 'Samantha:[EROTICA]Oooohhhh...[SEX SFX].',
 'John:[EROTICA]ugh, ugh, you like when I fuck that pussy dont you?',
 'Samantha:[EROTICA]yes! yes! uuh, uh! my tight cunt loves yoour big dick!']

### Step 2: Generate the audio for every speaker turn

In [22]:
pieces = []
silence = np.zeros(int(0.5*SAMPLE_RATE))
for line in script:
    if ":" in line:
        speaker, text = line.split(": ")
        pieces += [generate_audio(text, history_prompt=speaker_lookup[speaker]), silence.copy()]




ValueError: not enough values to unpack (expected 2, got 1)

### Step 3: Concatenate all of the audio and play it

In [12]:
from scipy.io import wavfile
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)
#wavfile.write("dialoguesex.wav", SAMPLE_RATE, Audio)

ValueError: need at least one array to concatenate